In [91]:
# == importar bibliotecas == 

from bs4 import BeautifulSoup as bs
import unicodedata as ucd
import requests
import pandas as pd
import numpy as np
import os
from itertools import chain 
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.by import By

In [92]:
# == settando variáveis == 

JORNAL = 'PDT'
LABEL = 'centro' 
URL = "https://pdt.org.br/index.php/artigos/"
NUM_PAG = 46
class_1 = 'inside-grid-column'
class_2 = 'entry-content'

In [93]:
# == configurando env == 

driver= webdriver.Chrome()
driver.maximize_window()

In [72]:
# == obter html == 

def get_soup(driver):
    soup = bs(driver.page_source, "html.parser")
    return soup

In [73]:
# == obter dataframe do texto == 

def return_df(url, text):
    return pd.DataFrame({
        'jornal': JORNAL,
        'url': url,
        'artigo': text,
        'label': LABEL
    }, index=[1])

In [ ]:
# == coletar os sites de artigos == 

def get_links(pag_num):
    cur_list = []
    driver.get(f'{URL}pagina/{pag_num}')
    sleep(5)
    soup = get_soup(driver)
    for noticia in soup.find_all("div", class_=class_1):
        for artigo in noticia.find_all("a", href=True):
            cur_list.append(artigo.get('href'))
    return cur_list

list_url = []
for n in range(NUM_PAG):
    list_url.append(get_links(n))

In [79]:
# == remover url duplicado == 

list_url = list(chain.from_iterable(list_url))
list_url = list(set(list_url))

In [ ]:
# == obter conteúdo da notícia == 

def get_artigo(url):
    driver.get(url)
    sleep(5)
    soup = get_soup(driver)
    list_text = []
    for div in soup.find_all("div", class_=class_2):
        for p in div.find_all("p"):
            list_text.append(ucd.normalize("NFKD", p.text))
        return return_df(url, " ".join(list_text))

res = []
for url in list_url:
    res.append(get_artigo(url))
scrap = pd.concat(res)

In [101]:
# == salvar o scrapping == 

scrap.to_parquet('../dataset/scrapping/PDT.parquet')
scrap.to_csv('../dataset/scrapping/PDT.csv')

In [95]:
scrap.shape

(450, 4)